# Feature Engineering 2: Dimensionality and PCA

In [110]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
import math
#from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

used = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

### Question 1

In [111]:
used.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [112]:
for i in range(len(used)):
    used.loc[i,'weighted_average'] = (used.loc[i,'AVG_MATH_4_SCORE']*used.loc[i,'GRADES_4_G']
                                      +used.loc[i,'AVG_READING_4_SCORE']*used.loc[i,'GRADES_4_G']
                                      +used.loc[i,'AVG_MATH_8_SCORE']*used.loc[i,'GRADES_8_G']
                                      +used.loc[i,'AVG_READING_8_SCORE']*used.loc[i,'GRADES_8_G'])/(2*used.loc[i,'GRADES_4_G']+2*used.loc[i,'GRADES_8_G'])


### Question 2

In [113]:
expend_df = used.iloc[:,[8,9,10,11,12,25]]

expend_df.corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,weighted_average
TOTAL_EXPENDITURE,1.000000,0.991347,0.993314,0.946110,0.932447,0.089937
INSTRUCTION_EXPENDITURE,0.991347,1.000000,0.976889,0.909340,0.896103,0.103678
SUPPORT_SERVICES_EXPENDITURE,0.993314,0.976889,1.000000,0.953091,0.917782,0.094103
OTHER_EXPENDITURE,0.946110,0.909340,0.953091,1.000000,0.921932,0.004678
CAPITAL_OUTLAY_EXPENDITURE,0.932447,0.896103,0.917782,0.921932,1.000000,0.009855
weighted_average,0.089937,0.103678,0.094103,0.004678,0.009855,1.000000


None of the expenditure variables are particularly correlated with test scores. However, instruction expenditures have the highest correlation.

Most of the variance in expenditures will be driven by the size of the state. I would expect measures of expenditure per student would be more closely correlated with test scores.

In [114]:
used.columns

Index(['PRIMARY_KEY', 'STATE', 'YEAR', 'ENROLL', 'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
       'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
       'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE',
       'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE', 'weighted_average'],
      dtype='object')

In [115]:
for i in range(len(used)):
    used.loc[i,'per_student_exp'] = used.loc[i,'TOTAL_EXPENDITURE']/used.loc[i,'GRADES_ALL_G']
    used.loc[i,'per_student_exp_ins'] = used.loc[i,'INSTRUCTION_EXPENDITURE']/used.loc[i,'GRADES_ALL_G']
    used.loc[i,'per_student_exp_support'] = used.loc[i,'SUPPORT_SERVICES_EXPENDITURE']/used.loc[i,'GRADES_ALL_G']
    used.loc[i,'per_student_exp_other'] = used.loc[i,'OTHER_EXPENDITURE']/used.loc[i,'GRADES_ALL_G']
    used.loc[i,'per_student_exp_capital'] = used.loc[i,'CAPITAL_OUTLAY_EXPENDITURE']/used.loc[i,'GRADES_ALL_G']
    
used.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,weighted_average,per_student_exp,per_student_exp_ins,per_student_exp_support,per_student_exp_other,per_student_exp_capital
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,208.327876,252.187522,207.963517,NaN,NaN,3.924726,2.191304,1.087052,NaN,0.257409
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,NaN,NaN,NaN,258.859712,NaN,8.657035,4.436391,3.123710,NaN,0.333387
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,215.253932,265.366278,206.212716,262.169895,235.639070,5.532095,2.335262,1.638906,NaN,0.990621
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,210.206028,256.312090,208.634458,264.619665,235.441283,4.301008,2.379523,1.193035,NaN,0.358319
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,208.398961,260.892247,196.764414,NaN,NaN,5.753273,3.044007,1.806386,NaN,0.433462


In [116]:
expend_df = used.iloc[:,[25,26,27,28,29,30]]

expend_df.corr()

,weighted_average,per_student_exp,per_student_exp_ins,per_student_exp_support,per_student_exp_other,per_student_exp_capital
weighted_average,1.000000,0.399690,0.455011,0.236486,0.141134,0.023968
per_student_exp,0.399690,1.000000,0.998711,0.997788,0.978446,0.968737
per_student_exp_ins,0.455011,0.998711,1.000000,0.995436,0.976244,0.960302
per_student_exp_support,0.236486,0.997788,0.995436,1.000000,0.976693,0.964319
per_student_exp_other,0.141134,0.978446,0.976244,0.976693,1.000000,0.948943
per_student_exp_capital,0.023968,0.968737,0.960302,0.964319,0.948943,1.000000


As expected, correlations are much higher. Instruction expenditures per student is still the most correlated with our weighted average of test scores.

### Question 3

Apply PCA to the 4 expenditure variables

In [117]:
# Drop NaN values
expend_df = expend_df.dropna()

# Normalize the data
# I also remove our target variable from our X matrix. This wasn't explicitly mentioned
# in the tutorial, but if the end goal is predicting our target variable from certain
# explanatory variables, it seems like it would be cheating/counterproductive to use
# our target variable in our components.
X = StandardScaler().fit_transform(expend_df.iloc[:,2:6])

# Transpose X
Xt = X.T

# Find covariance matrix of X
Cx = np.cov(Xt)

# Find Eigenvalues and Eigenvectors of the Covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(Cx)

# Find how much of variance is explained by each component
print(eig_val_cov / sum(eig_val_cov))

[0.58181917 0.03298999 0.16950112 0.21568972]


58% of the total variance is explained by the 1st component

### Question 4

What is the correlation between the weighted score variable and the 1st principal component?

In [118]:
sklearn_pca = PCA(n_components=1)

Ysklearn = sklearn_pca.fit_transform(X)

expend_df['comp1'] = Ysklearn

expend_df.columns

expend_df.corr()

,weighted_average,per_student_exp,per_student_exp_ins,per_student_exp_support,per_student_exp_other,per_student_exp_capital,comp1
weighted_average,1.000000,0.384523,0.441107,0.225516,0.141134,-0.001837,0.298205
per_student_exp,0.384523,1.000000,0.961233,0.933362,0.439845,0.442053,0.959832
per_student_exp_ins,0.441107,0.961233,1.000000,0.863565,0.399590,0.282871,0.901096
per_student_exp_support,0.225516,0.933362,0.863565,1.000000,0.426690,0.359400,0.926780
per_student_exp_other,0.141134,0.439845,0.399590,0.426690,1.000000,0.148900,0.626809
per_student_exp_capital,-0.001837,0.442053,0.282871,0.359400,0.148900,1.000000,0.513315
comp1,0.298205,0.959832,0.901096,0.926780,0.626809,0.513315,1.000000


The correlation is 0.298

### Question 5

Would you prefer to use per-student instruction expenditure or the 1st principal component?

In this case, per-student instruction expenditure has a stronger correlation with our target variable. If you have to choose 1 variable, it is still the most preferable to use.